## Transformerサンプルコード
論文データを学習させてナンチャッテ論文をlatex形式で出力させる<br>
Transformenr版

### ライブラリインポート

In [1]:
!pip install datasets tqdm torch requests feedparser --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 2.7 MB/s eta 0:00:00


In [2]:
import os
import tarfile
import glob
import re
import requests
from tqdm import tqdm
import urllib.parse
import feedparser
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

### データセット読み込み

#### arXiv API から論文 ID をまとめて取得する関数

In [3]:
def fetch_arxiv_ids(category="cs.LG", max_results=200):
    """
    指定カテゴリの新しい arXiv 論文 ID をまとめて取得する簡易関数
    例: category="cs.LG"（Machine Learning）
    """
    base_url = "http://export.arxiv.org/api/query"

    # APIのクエリ: カテゴリ指定 + 新しい順
    search_query = f"cat:{category}"
    params = {
        "search_query": search_query,
        "start": 0,
        "max_results": max_results,
        "sortBy": "submittedDate",
        "sortOrder": "descending",
    }

    url = base_url + "?" + urllib.parse.urlencode(params)
    print("🔍 arXiv API URL:", url)

    feed = feedparser.parse(url)

    paper_ids = []
    for entry in feed.entries:
        # entry.id 例: "http://arxiv.org/abs/2401.12345v1"
        m = re.search(r'arxiv.org/abs/(\d{4}\.\d+)', entry.id)
        if m:
            pid = m.group(1)  # "2401.12345"
            paper_ids.append(pid)

    print(f"📄 取得した論文ID数: {len(paper_ids)} 件")
    return paper_ids

# ここで好きなカテゴリ・件数を指定
# 例1: cs.LG (Machine Learning) 最新から200本
# 例2: math.PR (Probability)
paper_ids = fetch_arxiv_ids(category="cs.LG", max_results=200)

print(paper_ids)

🔍 arXiv API URL: http://export.arxiv.org/api/query?search_query=cat%3Acs.LG&start=0&max_results=200&sortBy=submittedDate&sortOrder=descending
📄 取得した論文ID数: 200 件
['2511.16674', '2511.16665', '2511.16661', '2511.16655', '2511.16652', '2511.16629', '2511.16622', '2511.16613', '2511.16599', '2511.16597', '2511.16596', '2511.16592', '2511.16587', '2511.16579', '2511.16575', '2511.16573', '2511.16571', '2511.16551', '2511.16550', '2511.16549', '2511.16543', '2511.16540', '2511.16527', '2511.16523', '2511.16520', '2511.16512', '2511.16501', '2511.16483', '2511.16482', '2511.16476', '2511.16475', '2511.16468', '2511.16467', '2511.16445', '2511.16430', '2511.16427', '2511.16426', '2511.16416', '2511.16398', '2511.16377', '2511.16375', '2511.16374', '2511.16373', '2511.16346', '2511.16340', '2511.16333', '2511.16318', '2511.16309', '2511.16297', '2511.16288', '2511.16287', '2511.16258', '2511.16231', '2511.16226', '2511.16225', '2511.16218', '2511.16216', '2511.16207', '2511.16204', '2511.16194'

#### arXiv ソースをダウンロード＆展開

In [4]:
os.makedirs("papers", exist_ok=True)

for pid in tqdm(paper_ids, desc="Downloading", ncols=80, ascii=True):
    url = f"https://arxiv.org/src/{pid}v1"
    out_path = f"papers/{pid}.tar.gz"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            with open(out_path, "wb") as f:
                f.write(r.content)
            # tar / tar.gz / その他の圧縮形式に柔軟に対応
            try:
                with tarfile.open(out_path, "r:*") as tar:
                    tar.extractall(f"papers/{pid}")
            except tarfile.ReadError:
                print(f"Skipped {pid} (not a tar file)")
        else:
            print(f"Skipped {pid} (status {r.status_code})")
    except Exception as e:
        print(f"Error downloading {pid}: {e}")


Downloading:   0%|                                      | 0/200 [00:00<?, ?it/s]/tmp/ipython-input-2326405988.py:14: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(f"papers/{pid}")
Downloading:  17%|####9                        | 34/200 [00:04<00:16, 10.26it/s]

Skipped 2511.16445 (not a tar file)


Downloading:  20%|#####8                       | 40/200 [00:04<00:14, 11.40it/s]

Skipped 2511.16416 (not a tar file)
Skipped 2511.16398 (not a tar file)


Downloading:  42%|############                 | 83/200 [00:09<00:14,  8.21it/s]

...

Downloading:  92%|#########################9  | 185/200 [00:20<00:01, 12.37it/s]

Skipped 2511.15112 (not a tar file)


Downloading:  96%|##########################7 | 191/200 [00:20<00:00,  9.63it/s]

Skipped 2511.15067 (not a tar file)
Skipped 2511.15062 (not a tar file)


Downloading: 100%|############################| 200/200 [00:22<00:00,  8.87it/s]


#### 結果確認

In [5]:
!find papers -name "*.tex" | head -n 20

papers/2511.16080/sections/c_subsections/performance_data.tex
papers/2511.16080/sections/c_subsections/row_time_data.tex
papers/2511.16080/sections/1_introduction.tex
papers/2511.16080/sections/3_incremental_resolutions.tex
papers/2511.16080/sections/2_ragged_and_named_dimensions.tex
papers/2511.16080/sections/6_related_work.tex
papers/2511.16080/sections/a_compatibility.tex
papers/2511.16080/sections/2_subsections/2B_shapes_and_coordinates.tex
papers/2511.16080/sections/2_subsections/2C_arrays.tex
papers/2511.16080/sections/2_subsections/2A_dimensions.tex
papers/2511.16080/sections/7_conclusion.tex
papers/2511.16080/sections/1_subsections/1B_challenges.tex
papers/2511.16080/sections/1_subsections/1C_our_design.tex
papers/2511.16080/sections/1_subsections/1D_contributions.tex
papers/2511.16080/sections/1_subsections/1A_motivating_example.tex
papers/2511.16080/sections/5_evaluation.tex
papers/2511.16080/sections/c_evaluation.tex
papers/2511.16080/sections/4_operon.tex
papers/2511.16080/

### データ加工

#### LaTeX 本文抽出 ＋ 前処理

In [6]:
def extract_latex_body(tex_text):
    """LaTeX本文を抽出してクリーン化"""
    text = re.sub(r'%.*', '', tex_text)  # コメント削除
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [7]:
import re, glob

all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
            cleaned = extract_latex_body(raw)  # ← あなたの定義した関数
            if len(cleaned) > 200:
                all_texts.append(cleaned)
                # print(f"抽出成功: {fname}")
    except Exception as e:
        print(f"{fname}: {e}")

print("抽出された論文本数:", len(all_texts))

抽出された論文本数: 817


In [8]:
# all_texts からトークンを作るところ（←これが先）

TOKEN_PATTERN = r'\\[a-zA-Z]+|[{}_^=+\-\*/()0-9a-zA-Z]+'
SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

tokens = []
for txt in all_texts:
    # 各論文ごとに <bos>, <eos> を挟んでトークナイズ
    body_tokens = re.findall(TOKEN_PATTERN, txt)
    tokens.extend(["<bos>"] + body_tokens + ["<eos>"])

print(f"トークン総数: {len(tokens):,}")

トークン総数: 1,986,646


In [9]:
def extract_latex_body(tex_text: str) -> str:
    """LaTeX本文を抽出してある程度クリーンアップする（簡易版）"""
    # コメント行を削除（% 以降）
    text = re.sub(r'%.*', '', tex_text)

    # \begin{document}〜\end{document} の間だけを抜き出し
    m = re.search(r'\\begin{document}(.*?)\\end{document}', text, re.DOTALL)
    if m:
        text = m.group(1)

    # bibliography / 参考文献・付録以降をざっくり削る（必要に応じて調整）
    text = re.split(r'\\bibliography|\\begin{thebibliography}', text)[0]
    text = re.split(r'\\appendix', text)[0]

    # 連続する空白を1つにまとめる
    text = re.sub(r'\s+', ' ', text).strip()
    return text

all_texts = []
for fname in glob.glob("papers/**/*.tex", recursive=True):
    try:
        with open(fname, encoding="utf-8", errors="ignore") as f:
            raw = f.read()
        cleaned = extract_latex_body(raw)
        if len(cleaned) > 200:  # ごく短いファイルはノイズなので除外
            all_texts.append(cleaned)
            #print(f"xtracted from: {fname}")
    except Exception as e:
        print(f"{fname}: {e}")

print(f"\n抽出された論文本数: {len(all_texts)}")
print(f"総テキスト長: {sum(len(t) for t in all_texts):,} 文字")

# テキストを一つにまとめたファイルも保存しておく（デバッグ用）
merged_text = "\n\n".join(all_texts)
with open("merged_corpus.tex", "w", encoding="utf-8") as f:
    f.write(merged_text)
print("\nmerged_corpus.tex に保存しました")


抽出された論文本数: 796
総テキスト長: 10,881,347 文字

merged_corpus.tex に保存しました


#### トークン化と語彙辞書作成

In [10]:
from collections import Counter

# SPECIAL_TOKENS はさっきと同じものを使う
# SPECIAL_TOKENS = ["<unk>", "<bos>", "<eos>"]

print(f"トークン総数: {len(tokens):,}")

# 語彙を頻度上位だけに圧縮
counter = Counter(tokens)

VOCAB_LIMIT = 40000
most_common_tokens = [
    w for (w, c) in counter.most_common(VOCAB_LIMIT)
    if w not in SPECIAL_TOKENS
]

vocab = SPECIAL_TOKENS + most_common_tokens

word2idx = {w: i for i, w in enumerate(vocab)}
idx2word = {i: w for i, w in enumerate(vocab)}

UNK_IDX = word2idx["<unk>"]
BOS_IDX = word2idx["<bos>"]
EOS_IDX = word2idx["<eos>"]

print(f"語彙数（圧縮後）: {len(vocab):,}")

# トークン列を index に変換
indexed_tokens = [word2idx.get(t, UNK_IDX) for t in tokens]

# 念のためチェック
max_id = max(indexed_tokens)
print("最大トークンID:", max_id)
print("語彙サイズ:", len(vocab))
assert max_id < len(vocab), "トークンIDが語彙サイズを超えています！"


トークン総数: 1,986,646
語彙数（圧縮後）: 40,001
最大トークンID: 40000
語彙サイズ: 40001


### データセット定義

#### Datasetクラス定義

In [11]:
class LatexDataset(Dataset):
    """大きなトークン列から、ランダムにサンプルを切り出す Dataset"""
    def __init__(self, data, seq_length=60, samples_per_epoch=3000):
        self.data = data                      # ← すでに index 化された列
        self.seq_length = seq_length
        self.samples_per_epoch = samples_per_epoch

        self.max_start = len(self.data) - (self.seq_length + 1)
        assert self.max_start > 0, "データが短すぎます"

    def __len__(self):
        # 1epochあたりのサンプル数を直接決める
        return self.samples_per_epoch

    def __getitem__(self, idx):
        # idx は使わず、毎回ランダムな位置から切り出す
        start = random.randint(0, self.max_start)
        end = start + self.seq_length + 1
        chunk = self.data[start:end]

        x = torch.tensor(chunk[:-1], dtype=torch.long)  # [T]
        y = torch.tensor(chunk[1:], dtype=torch.long)   # [T]
        return x, y


In [31]:
# ここで初めて dataset を作る（tokens ではなく indexed_tokens を渡す）
seq_length = 128
samples_per_epoch = 8000
dataset = LatexDataset(indexed_tokens, seq_length=seq_length,
                       samples_per_epoch=samples_per_epoch)


#### Train / Validation に分割して DataLoader を作成

In [32]:
train_ratio = 0.9
n_total = len(dataset)
n_train = int(n_total * train_ratio)
n_val = n_total - n_train

train_ds, val_ds = random_split(dataset, [n_train, n_val])

batch_size = 32

train_loader = DataLoader(
    train_ds,
    batch_size=batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_ds,
    batch_size=batch_size,
    shuffle=False,
    drop_last=False,
    num_workers=2,
    pin_memory=True
)

print(f"データ数: total={n_total}, train={len(train_ds)}, val={len(val_ds)}")
print(f"1epoch のステップ数（train）: {len(train_loader)}")


データ数: total=8000, train=7200, val=800
1epoch のステップ数（train）: 225


### 学習

#### Transformerモデル定義(疑似数学論文生成)

In [33]:
# Transformerモデル定義(疑似数学論文生成)
class LatexTransformer(nn.Module):
    """
    LaTeX トークン列用のシンプルな Transformer 言語モデル
    入力: x [B, T]  (トークンID)
    出力: logits [B, T, V]  (各位置ごとの単語分布のロジット)
    """
    def __init__(self,
                 vocab_size: int,
                 d_model: int = 384,
                 nhead: int = 6,
                 num_layers: int = 6,
                 dim_feedforward: int = 1024,
                 dropout: float = 0.1,
                 max_seq_len: int = 512):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        # 語彙埋め込み
        self.token_embedding = nn.Embedding(vocab_size, d_model)
        # 位置埋め込み（シンプルな learnable embedding）
        self.pos_embedding = nn.Embedding(max_seq_len, d_model)
        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True  # [B, T, E] で処理
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers
        )
        # 出力層
        self.fc_out = nn.Linear(d_model, vocab_size)
        # Causal mask をキャッシュ用に保持
        self.register_buffer("mask", None, persistent=False)

    def _generate_square_subsequent_mask(self, sz: int, device: torch.device):
        """
        自己回帰用のマスク (未来のトークンを見ないようにする)
        shape: [T, T]
        """
        if (self.mask is None) or (self.mask.size(0) != sz):
            mask = torch.full((sz, sz), float("-inf"), device=device)
            mask = torch.triu(mask, diagonal=1)
            self.mask = mask
        return self.mask

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        x: [B, T]  トークンID列
        戻り値:
            logits: [B, T, V]
        """
        B, T = x.size()
        device = x.device

        # 位置ID [0, 1, ..., T-1]
        positions = torch.arange(T, device=device).unsqueeze(0).expand(B, T)  # [B, T]

        tok_emb = self.token_embedding(x)          # [B, T, d_model]
        pos_emb = self.pos_embedding(positions)    # [B, T, d_model]
        h = tok_emb + pos_emb                      # [B, T, d_model]

        # causal mask (未来の情報を見ない)
        src_mask = self._generate_square_subsequent_mask(T, device=device)  # [T, T]

        # TransformerEncoder に通す
        # batch_first=True なので h: [B, T, d_model], mask は [T, T]
        h = self.transformer(h, mask=src_mask)     # [B, T, d_model]

        logits = self.fc_out(h)                    # [B, T, V]
        return logits


#### 繰り返し計算

In [ ]:
%%time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

vocab_size = len(vocab)
model = LatexTransformer(
    vocab_size=vocab_size,
    d_model=384,
    nhead=6,
    num_layers=6,
    dim_feedforward=1024,
    dropout=0.1,
    max_seq_len=512       # seq_length より十分大きければOK
).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

num_epochs = 30
max_grad_norm = 1.0  # 勾配クリッピングの最大ノルム

for epoch in range(num_epochs):
    # -------------------------------
    # Train
    # -------------------------------
    model.train()
    total_train_loss = 0.0

    for x, y in train_loader:
        # x, y: [B, T]
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)  # [B, T, V]

        # CrossEntropyLoss は [N, C] vs [N] をとるので reshape する
        loss = criterion(
            logits.view(-1, logits.size(-1)),  # [B*T, V]
            y.view(-1)                         # [B*T]
        )
        loss.backward()

        # 勾配クリッピング（勾配爆発対策）
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

        optimizer.step()
        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_loader)

    # -------------------------------
    # Validation
    # -------------------------------
    model.eval()
    total_val_loss = 0.0
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            val_loss = criterion(
                logits.view(-1, logits.size(-1)),
                y.view(-1)
            )
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)

    # 学習率スケジューラを進める
    scheduler.step()

    print(
        f"Epoch [{epoch+1}/{num_epochs}] "
        f"train_loss={avg_train_loss:.4f}  val_loss={avg_val_loss:.4f}  "
        f"lr={scheduler.get_last_lr()[0]:.5f}"
    )


device: cuda
Epoch [1/30] train_loss=7.9218  val_loss=7.7958  lr=0.00100
Epoch [2/30] train_loss=7.7953  val_loss=7.7725  lr=0.00100
Epoch [3/30] train_loss=7.7709  val_loss=7.7349  lr=0.00100
Epoch [4/30] train_loss=7.7522  val_loss=7.7479  lr=0.00100
Epoch [5/30] train_loss=7.7509  val_loss=7.7648  lr=0.00100
Epoch [6/30] train_loss=7.7471  val_loss=7.7500  lr=0.00100
Epoch [7/30] train_loss=7.7503  val_loss=7.7449  lr=0.00100


### 予測

#### サンプリング用ヘルパー




In [28]:
def sample_from_logits(logits: torch.Tensor,
                       temperature: float = 0.8,
                       top_k: int = 50) -> int:
    """
    logits: [V] の1時刻分のロジット
    戻り値: 次に選ぶトークンID
    """
    # 温度でスケーリング
    logits = logits / temperature

    # top-k だけ残してサンプリング（極端なノイズを減らす）
    if top_k is not None and top_k > 0:
        values, indices = torch.topk(logits, top_k)
        probs = torch.softmax(values, dim=-1)
        idx_in_topk = torch.multinomial(probs, 1).item()
        next_token_id = indices[idx_in_topk].item()
        return next_token_id
    else:
        probs = torch.softmax(logits, dim=-1)
        next_token_id = torch.multinomial(probs, 1).item()
        return next_token_id


#### テキスト生成関数

In [29]:
def generate(seed_text, length=200, temperature=0.8, top_k=50, seq_length=60):
    model.eval()
    words = re.findall(TOKEN_PATTERN, seed_text)

    for _ in range(length):
        # 🔥 直近 seq_length 分だけ使う
        seq = [word2idx.get(w, UNK_IDX) for w in words[-seq_length:]]

        x = torch.tensor([seq], dtype=torch.long).to(device)

        with torch.no_grad():
            logits = model(x)[0, -1]  # ラスト時間ステップのみ

            # <unk> を出にくくする
            logits[UNK_IDX] -= 10.0

            logits = logits / temperature

            # top-k サンプリング（推奨）
            values, indices = torch.topk(logits, top_k)
            probs = torch.softmax(values, dim=0)
            next_idx = indices[torch.multinomial(probs, 1).item()].item()

        words.append(idx2word[next_idx])

    return " ".join(words)


#### テキスト生成サンプル

In [30]:
seed = r"\section{Introduction} We consider the problem of minimizing"
print("\n生成結果:\n")
text = generate(seed, length=200, temperature=0.7, top_k=30, seq_length=seq_length)
print(text)


生成結果:

\section {Introduction} We consider the problem of minimizing a collection of a dataset of the dataset \mathcal {D} and consisting of the dataset \mathcal {D} R \mathcal R \mathcal {X} \rightarrow \mathcal {C} \mathcal {D} R \mathcal {F}) \to \mathcal {D} R \mathcal {F}) \to \mathcal {F} \setminus {0 } \mathcal {F} \to \mathcal {C} \mid \mathcal {F} \rightharpoonup \mathcal {F}^ \mathcal {D} } \subseteq \mathcal {D} } \mathcal {C}( \mathcal {D} } \mathcal {F} \setminus \mathcal {F} ) = \mathcal {C}( \mathcal {D} R \mathcal {F}) \rightharpoonup \mathcal {F}) ) ( \mathcal {E}^ \mathcal {D} R \mathcal {F} \mathcal {F}) \to \emptyset ) is said to \mathcal {F} \rightharpoonup \mathbb {N}_0 ) that ( \mathcal {C}( \mathcal {D} R \mathcal {F}) ) = \mathbb {P}_{ \geq p}( \mathcal {D} ) \end {definition} \subsection {Discussion of Theorem \ref {thm canonical_expansions}-(1) The following proposition holds \begin {theorem} \label {thm canonical_expansions}-(1) Let \mathcal {D} = \mathbb {

In [19]:
# =====================================================
# LaTeX → PDF 変換 (Google Colab / Linux 環境用)
# =====================================================

import os, subprocess

# ① 生成済みLaTeX本文をここに貼る（整形後のtextを入れる）
latex_code = r"""
\documentclass[11pt]{article}

\usepackage{amsmath, amssymb, amsfonts}
\usepackage{bm}
\usepackage[margin=25mm]{geometry}
\usepackage{natbib}

\begin{document}

\begin{abstract}
We consider the problem of minimizing a loss function over a complex
training space. Our method aims to avoid overfitting on a single
graph-structured dataset and to provide a unified framework that can
handle standard and kernel-based models. We also discuss connections
to speculative inference methods~\citep{Li2021SpeculativeInference}
and show how our approach can be applied across different domains.
\end{abstract}

\section{Introduction}

We consider the problem of minimizing a learning objective over a
training distribution while maintaining stable performance on a test
distribution. Our model is trained across multiple domains and is
designed to improve interpretability and robustness of the learned
representations.

In the simplest setting, let $\theta$ denote the model parameters and
$\ell(\theta)$ be the empirical loss on the training set. A generic
gradient-based update can be written as
\begin{equation}
  \theta_{t+1}
  = \theta_t - \eta \, \frac{\partial \ell(\theta_t)}{\partial \theta},
\end{equation}
where $\eta > 0$ is the learning rate. Although the automatically
generated text contained an incomplete expression, the above equation
represents a valid example consistent with the original intent.

We also assume the existence of a kernel-based representation
$K(x, x')$ on the input space. Let $\mathcal{R}(\theta)$ denote a
regularization term, and consider the regularized objective
\begin{equation}
  \mathcal{L}(\theta)
  = \ell(\theta) + \lambda \, \mathcal{R}(\theta),
\end{equation}
where $\lambda \ge 0$ controls the strength of regularization.
Under suitable assumptions, one can show that $\mathcal{L}$ admits
a finite-sample bound that depends on the complexity of the
underlying kernel space.

\section{Conclusion}

The sequence produced by the Transformer model has been rewritten
into a compilable \LaTeX{} document. While the original output mixed
incomplete equations and environment mismatches, we replaced them
with consistent mathematical expressions and a coherent structure
consisting of an abstract, an introduction, and basic equations.

\bibliographystyle{plainnat}
\bibliography{references}

\end{document}
"""

# ② texファイルとして保存
with open("generated.tex", "w", encoding="utf-8") as f:
    f.write(latex_code)

# ③ LaTeXコマンドが使えるようにTexLiveをインストール
!apt-get update -qq
!apt-get install -y texlive-latex-base texlive-latex-extra texlive-fonts-recommended > /dev/null

# ④ pdflatexでPDFに変換
!pdflatex -interaction=nonstopmode generated.tex > /dev/null

# ⑤ 出力確認
print("PDF生成完了: generated.pdf")
!ls -lh generated.pdf


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Extracting templates from packages: 100%
PDF生成完了: generated.pdf
-rw-r--r-- 1 root root 85K Nov 23 06:39 generated.pdf
